## Delete: 내용 중복/Unique한 값이 한개/gut,human 이 아닌 값 삭제 

1. geo 칼럼 전처리 

In [3]:
import pandas as pd

# 1. CSV 불러오기
df = pd.read_csv("all_column.csv")

# 2. 필요한 컬럼 복사
cols = ["lat_lon", "geographic location (latitude)", "geographic location (longitude)"]
df_coords = df[cols].copy()

# 3. lat_lon 문자열 파싱 함수
def parse_lat_lon(latlon_str):
    if pd.isna(latlon_str):
        return None, None
    try:
        parts = latlon_str.strip().split()
        lat = float(parts[0]) * (-1 if parts[1] == 'S' else 1)
        lon = float(parts[2]) * (-1 if parts[3] == 'W' else 1)
        return lat, lon
    except:
        return None, None

# 4. 숫자 좌표 없을 때 lat_lon에서 추출
df_coords[["lat_from_str", "lon_from_str"]] = df_coords.apply(
    lambda row: pd.Series(parse_lat_lon(row["lat_lon"]))
    if pd.isna(row["geographic location (latitude)"]) or pd.isna(row["geographic location (longitude)"])
    else pd.Series([None, None]),
    axis=1
)

# 5. 최종 좌표 결합
def combine_coordinates(row):
    lat = row["geographic location (latitude)"] if not pd.isna(row["geographic location (latitude)"]) else row["lat_from_str"]
    lon = row["geographic location (longitude)"] if not pd.isna(row["geographic location (longitude)"]) else row["lon_from_str"]
    if pd.isna(lat) or pd.isna(lon):
        return None
    return f"({lat:.6f}, {lon:.6f})"

df_coords["geo_coord"] = df_coords.apply(combine_coordinates, axis=1)

# 6. 원본 df에 geo_coord 붙이기
df["geo_coord"] = df_coords["geo_coord"]

# 7. 사용한 세 컬럼 삭제
df.drop(columns=["lat_lon", "geographic location (latitude)", "geographic location (longitude)"], inplace=True)

# 8. CSV 저장
df.to_csv("all_column_geo_coord1.csv", index=False)


2. not human gut 
<scientific_name> or <organism> 의 값이 → laboratory control인 행을 제거
<scientific_name> or <organism> 의 값이 → blood metagenome 인 행 삭제
<title> 의 nasal wash solution Saline → 에 해당하는 row는 삭제

In [4]:
import pandas as pd

# CSV 불러오기
df = pd.read_csv("all_column_geo_coord1.csv")

# 소문자로 변환
df["scientific_name"] = df["scientific_name"].astype(str).str.lower()
df["organism"] = df["organism"].astype(str).str.lower()
df["title"] = df["title"].astype(str).str.lower()

# 제거 조건 정의
drop_conditions = (
    (df["scientific_name"] == "laboratory control") |
    (df["organism"] == "laboratory control") |
    (df["scientific_name"] == "blood metagenome") |
    (df["organism"] == "blood metagenome") |
    (df["title"] == "nasal wash solution saline")
)

# 조건에 해당하지 않는 행만 남김
df_filtered = df[~drop_conditions]

# 결과 저장
df_filtered.to_csv("all_column_nothumangut2.csv", index=False)


In [5]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv("all_column_nothumangut2.csv")  # 역슬래시는 경로 오류 유발 가능하므로 슬래시(/) 사용

# title 컬럼의 고유값 추출
unique_titles = df["title"].dropna().unique()

# 결과 출력
print(f"Unique title count: {len(unique_titles)}")
for title in unique_titles:
    print(title)


Unique title count: 343
gut patient
gut control
c0001 baseline
c0005 baseline
c0015 baseline
c0021 baseline
c0026 baseline
c0047 baseline
c0051 baseline
c0054 baseline
c0074 baseline
c0080 baseline
c0085 baseline
c0088 baseline
c0096 baseline
c0098 baseline
c0107 baseline
c0134 baseline
c0135 baseline
p0004 baseline
p0008 baseline
p0009 baseline
p0011 baseline
p0016 baseline
p0018 baseline
p0019 baseline
p0020 baseline
p0024 baseline
p0031 baseline
p0046 baseline
p0047 baseline
p0053 baseline
p0056 baseline
p0061 baseline
p0062 baseline
p0066 baseline
p0067 baseline
p0068 baseline
p0070 baseline
p0071 baseline
p0074 baseline
p0105 baseline
p0116 baseline
p0120 baseline
c0007 followup
c0015 followup
c0018 followup
c0019 followup
c0024 followup
c0026 followup
c0034 followup
c0048 followup
c0049 followup
c0051 followup
c0065 followup
c0074 followup
c0076 followup
c0082 followup
c0085 followup
c0089 followup
c0090 followup
c0095 followup
c0111 followup
c0116 followup
c0123 followup
c0134 f

3. 
**<title 칼럼의 row 행 삭제>**
titlepp
1. <위에서 함> nasal wash solution Saline -> 에 해당하는 row는 삭제
2. PCRblank, Blank-#, Lambda-#, no template control, mock-extraction control, reagent control 같은 실험적 음성 대조군에 해당하는 row 삭제
3. MIMARKS Survey related sample from human gut metagenome, Human sample from Homo sapiens -> human gut metagenome 이렇게 바꾸기, 용어 통일

In [6]:
import pandas as pd

# CSV 불러오기
df = pd.read_csv("all_column_nothumangut2.csv")

# 조건 1: nasal wash solution Saline 제거
# df = df[df["title"] != "nasal wash solution Saline"]

# 조건 2: 실험적 음성 대조군 제거
negative_control_keywords = [
    "PCRblank", "Blank-", "Lambda-", "no template control",
    "mock-extraction control", "reagent control"
]

# 해당 키워드가 포함된 title 제거
pattern = '|'.join(negative_control_keywords)
df = df[~df["title"].str.contains(pattern, case=False, na=False)]

# 조건 3: 특정 title 값을 human gut metagenome 으로 통일
replace_values = [
    "mimarks survey related sample from human gut metagenome",
    "feces",
    "human sample from homo sapiens",
    "mimarks survey related sample from human gut metagenome of generous donor"
]
df["title"] = df["title"].replace(replace_values, "human gut metagenome")

# 결과 저장 (선택)
df.to_csv("all_column_titlepp3.csv", index=False)

# 결과 확인용 출력
print("정제 후 title unique 값 수:", df["title"].nunique())


정제 후 title unique 값 수: 269


4.
environment (material) 칼럼 삭제
description 칼럼 삭제
investigation type 칼럼 삭제 


In [7]:
import pandas as pd

# CSV 불러오기
df = pd.read_csv("all_column_titlepp3.csv")

# 삭제할 컬럼 목록
cols_to_drop = ["environment (material)", "description", "investigation type"]

# 컬럼 삭제
df = df.drop(columns=cols_to_drop, errors="ignore")

# 결과 저장
df.to_csv("all_column_deleteduplicolumn4.csv", index=False)


In [8]:
import pandas as pd

# CSV 불러오기
df = pd.read_csv("all_column_deleteduplicolumn4.csv")

# 추출할 컬럼 목록
columns_of_interest = ["title","scientific_name", "organism", "host", "common_name"]

# 각 컬럼별 unique 값 출력
for col in columns_of_interest:
    uniques = df[col].dropna().unique()
    print(f"\n--- {col} (Unique count: {len(uniques)}) ---")
    for val in uniques:
        print(val)



--- title (Unique count: 269) ---
gut patient
gut control
c0001 baseline
c0005 baseline
c0015 baseline
c0021 baseline
c0026 baseline
c0047 baseline
c0051 baseline
c0054 baseline
c0074 baseline
c0080 baseline
c0085 baseline
c0088 baseline
c0096 baseline
c0098 baseline
c0107 baseline
c0134 baseline
c0135 baseline
p0004 baseline
p0008 baseline
p0009 baseline
p0011 baseline
p0016 baseline
p0018 baseline
p0019 baseline
p0020 baseline
p0024 baseline
p0031 baseline
p0046 baseline
p0047 baseline
p0053 baseline
p0056 baseline
p0061 baseline
p0062 baseline
p0066 baseline
p0067 baseline
p0068 baseline
p0070 baseline
p0071 baseline
p0074 baseline
p0105 baseline
p0116 baseline
p0120 baseline
c0007 followup
c0015 followup
c0018 followup
c0019 followup
c0024 followup
c0026 followup
c0034 followup
c0048 followup
c0049 followup
c0051 followup
c0065 followup
c0074 followup
c0076 followup
c0082 followup
c0085 followup
c0089 followup
c0090 followup
c0095 followup
c0111 followup
c0116 followup
c0123 follo

5.
scientific_name 칼럼 삭제
organism 칼럼 남기기 

-> column unique 한 값으로
homo sapiens
human gut metagenome
gut metagenome
의 값을 갖고 있는데 위의 3개의 값을 -> human gut metagenome 으로 통일 처리

In [9]:
import pandas as pd

# CSV 불러오기
df = pd.read_csv("all_column_deleteduplicolumn4.csv")

# organism 컬럼 값 통일
df["organism"] = df["organism"].astype(str).str.strip().str.lower()
df["organism"] = df["organism"].replace({
    "homo sapiens": "human gut metagenome",
    "gut metagenome": "human gut metagenome",
    "human gut metagenome": "human gut metagenome"  # 명시적으로 지정
})

# 불필요한 컬럼 삭제
cols_to_drop = ["scientific_name", "host", "common_name"]
df = df.drop(columns=[col for col in cols_to_drop if col in df.columns])

# 저장
df.to_csv("all_column_delete5.csv", index=False)


6.
collection_method 제거
수집 방법에 대해 -> 잠재적 confounder이긴 하지만, 내용이 너무 방대하기 때문에 일단 삭제하고, 이후 분석 전략으로 가져가기 

number 제거
쓸데없는 값 (1개의 프로젝트 내에서 구분하기 위해 쓰인걸로 보임)

In [10]:
import pandas as pd

# CSV 불러오기
df = pd.read_csv("all_column_delete5.csv")

# 1. collection_method 컬럼 제거
# → 수집 방법은 confounder로 작용 가능하지만, 값의 종류가 방대하여 일단 삭제하고 후속 분석에서 전략적으로 다룰 예정
# 2. number 컬럼 제거
# → 하나의 프로젝트 내에서 샘플 구분용으로 쓰인 것으로 보이며, 분석에 의미 없음
cols_to_drop = ["collection_method", "number"]
df = df.drop(columns=[col for col in cols_to_drop if col in df.columns])

# 저장
df.to_csv("all_column_delete6.csv", index=False)


<수동> all_column_delete7.csv

1. geo_nation 뒤로 geo_coord 칼럼 옮김 
2. title의 초반 parkinson 병 유무를 parkinson 칼럼에 옮기기 

host body product
environment (biome)
INSDC secondary accession
NCBI submission package	
env_material
env_biome	
BioSampleModel	
source_material_id	
isolation_source	
isolate
tissue
biomaterial_provider
env_local_scale
env_medium


생명 전공인 시정님이나 랩실분들께 삭제여부에 대해 묻기
- unique value 값과 결측치 비율 보여주기

In [11]:
import pandas as pd

# CSV 불러오기
df = pd.read_csv("all_column_delete7.csv")

# 대상 컬럼 목록
target_cols = [
    "host body product",
    "environment (biome)",
    "INSDC secondary accession",
    "NCBI submission package",
    "env_material",
    "env_biome",
    "BioSampleModel",
    "source_material_id",
    "isolation_source",
    "isolate",
    "tissue",
    "biomaterial_provider",
    "env_local_scale",
    "env_medium"
]

# 각 컬럼에 대해 unique 값과 개수 출력
for col in target_cols:
    if col in df.columns:
        unique_vals = df[col].dropna().unique()
        print(f"--- {col} (Unique count: {len(unique_vals)}) ---")
        for val in unique_vals:
            print(val)
        print("\n")
    else:
        print(f"[!] Column not found: {col}\n")


--- host body product (Unique count: 1) ---
stool


--- environment (biome) (Unique count: 1) ---
terrestrial biome


--- INSDC secondary accession (Unique count: 522) ---
SRS2111883
SRS2111882
SRS2111886
SRS2111893
SRS2111894
SRS2111900
SRS2111902
SRS2111914
SRS2111920
SRS2111923
SRS2111924
SRS2111925
SRS2111926
SRS2111928
SRS2111933
SRS2111937
SRS2111940
SRS2111941
SRS2111942
SRS2111884
SRS2111888
SRS2111891
SRS2111944
SRS2111895
SRS2111896
SRS2111897
SRS2111947
SRS2111899
SRS2111901
SRS2111950
SRS2111951
SRS2111952
SRS2111909
SRS2111957
SRS2111958
SRS2111913
SRS2111916
SRS2111918
SRS2111919
SRS2111962
SRS2111921
SRS2111963
SRS2111930
SRS2111964
SRS2111967
SRS2111968
SRS2111969
SRS2111970
SRS2111971
SRS2111975
SRS2111980
SRS2111983
SRS2111986
SRS2111988
SRS2111991
SRS2111996
SRS2111995
SRS2111999
SRS2112002
SRS2112006
SRS2112008
SRS2112010
SRS2112012
SRS2112018
SRS2112020
SRS2112022
SRS2112030
SRS2112032
SRS2112033
SRS2112035
SRS2112038
SRS2112042
SRS2112050
SRS2112051
SRS2112052
SRS

In [12]:
import pandas as pd

# CSV 불러오기
df = pd.read_csv("all_column_delete7.csv")

# 대상 컬럼 목록
target_cols = [
    "host body product",
    "environment (biome)",
    "INSDC secondary accession",
    "NCBI submission package",
    "env_material",
    "env_biome",
    "BioSampleModel",
    "source_material_id",
    "isolation_source",
    "isolate",
    "tissue",
    "biomaterial_provider",
    "env_local_scale",
    "env_medium"
]

# 결측치 비율 계산 및 출력
print("--- Missing Value Ratio by Column ---")
for col in target_cols:
    if col in df.columns:
        missing_ratio = df[col].isna().mean()
        print(f"{col}: {missing_ratio:.2%}")
    else:
        print(f"[!] Column not found: {col}")


--- Missing Value Ratio by Column ---
host body product: 83.08%
environment (biome): 83.08%
INSDC secondary accession: 66.79%
NCBI submission package: 66.79%
env_material: 85.24%
env_biome: 85.24%
BioSampleModel: 20.67%
source_material_id: 100.00%
isolation_source: 100.00%
isolate: 87.28%
tissue: 87.28%
biomaterial_provider: 87.28%
env_local_scale: 53.88%
env_medium: 53.88%


### 8. 칼럼 제거 



source_material_id 칼럼 제거
isolation_source 칼럼 제거

host body product 칼럼 제거
environment (biome) 칼럼 제거 

INSDC secondary accession 칼럼 제거
isolate 칼럼 제거
tissue 칼럼 제거
biomaterial_provider 칼럼 제거
env_local_scale 칼럼 제거
env_medium 칼럼 제거

NCBI submission package 칼럼 제거 
env_material 는 값이 oronasal secretion 인 경우의 row를 카운팅해줘
env_biome 는 값이 nasal cavity 인 경우의 row를 카운팅 해줘
BioSampleModel 칼럼 제거


1. 결측률 100% : source_material_id, isolation_source 제거
2. host body product - (샘플이 채취된 신체 산물) - stool 단일값 - 83.08%	-> 제거(정보 불균형 심하고 단일값)
3. environment (biome) - (환경 분류) - terrestrial biome 단일값 - 83.08%   -> 제거 가능 – 정보성 없음
4. INSDC secondary accession - (Sample Run ID의 서브 식별자) - 고유값 많음 (522개) - 66.79%	 -> id 값의 다른 식별자가 있으므로 제거
5. isolate - (샘플 구분 ID (internal)) - 190개 값, 의미 불명 - 87.28%	            ->  제거 권장 – 의미 불명, 정보 혼란 유발
6. tissue - (시료가 유래한 인체 조직 부위) - stool (거의 단일값)	                    -> 장내 샘플이면 중복정보 (redundant) 제거
7. biomaterial_provider	(생체자원 제공 기관) - TAH_UM (거의 단일값)	                -> 프로젝트 정보에 종속됨 (분석 목적과 무관)	제거
8. env_local_scale - (환경의 미시적 위치(예: 방 안, 농장 내 위치 등)) - not applicable (단일값)	    -> 비의미적 (전부 NA)	제거
9. env_medium - (환경 내 존재하는 물질) - fecal material (거의 단일값)	                    -> tissue와 중복 정보	제거

10. NCBI submission package - (샘플을 NCBI에 제출할 때 사용하는 메타데이터 템플릿 유형.) -> 제거
11. env_material - (샘플이 어떤 물리적 물질(material)에서 유래했는지를 나타냄.) 
stool
oronasal secretion → row 제거 후 칼럼 제거 

12. env_biome - (샘플의 생물군계 (biome) 분류)
human gut
nasal cavity → row 제거 후 칼럼 제거 

13. BioSampleModel -> 결측치 20퍼지만 삭제 : 유의미한 정보를 못얻을 거 같음. 
MIMARKS.survey
Metagenome or environmental
Human
MIGS/MIMS/MIMARKS.human-gut

In [13]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv("all_column_delete7.csv")

# 소문자, 공백 제거 후 조건에 맞는 row 제거
df["env_material"] = df["env_material"].astype(str).str.strip().str.lower()
df["env_biome"] = df["env_biome"].astype(str).str.strip().str.lower()

# 조건에 맞는 행 제거
df = df[~(df["env_material"] == "oronasal secretion")]
df = df[~(df["env_biome"] == "nasal cavity")]

# 삭제할 컬럼 목록
columns_to_drop = [
    "source_material_id", "isolation_source", "host body product", 
    "environment (biome)", "INSDC secondary accession", "isolate", 
    "tissue", "biomaterial_provider", "env_local_scale", "env_medium", 
    "NCBI submission package", "env_material", "env_biome", "BioSampleModel"
]

# 존재하는 컬럼만 삭제
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

# 결과 저장
df.to_csv("all_column_delete8.csv", index=False)


9. 칼럼 삭제

timepoint 칼럼 -> title 과 중복이어서 삭제 
pcr primers 칼럼은 16s 시퀀싱이라는 것을 알려주는 지표 -> 이미 알고 잇는 내용으로 삭제

In [17]:
import pandas as pd

# CSV 불러오기
df = pd.read_csv("all_column_delete8.csv")

# 삭제할 칼럼 목록
columns_to_drop = ["timepoint", "pcr primers"]

# 존재하는 컬럼만 삭제
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

# 결과 저장
df.to_csv("all_column_delete9.csv", index=False)


10. 설문 칼럼 - unknown 으로 채우기

In [14]:
import pandas as pd

# CSV 불러오기
df = pd.read_csv("all_column_delete7.csv")

# 대상 컬럼 목록
target_cols = [
    "Sleep_aid", "Pain_med", "Ulcer_past_3_months", "SIBO", "Day_of_stool_collection_digestion_issue",
    "Cholesterol_med", "Antibiotics_current", "Constipation", "How_often_do_you_eat_NUTS",
    "Indigestion_drugs", "Asthma_or_COPD_med", "Thyroid_med", "Diarrhea", "GI_cancer_past_3_months",
    "Anti_inflammatory_drugs", "Day_of_stool_collection_abdominal_pain", "Day_of_stool_collection_excess_gas",
    "Co_Q_10", "Do_you_drink_alcohol", "Radiation_Chemo", "Blood_thinners", "Diabetes_med", "Laxatives",
    "Colitis", "Depression_anxiety_mood_med", "Birth_control_or_estrogen", "Blood_pressure_med",
    "Day_of_stool_collection_constipation", "Loss_10lbs_in_last_year", "Do_you_smoke", "IBD",
    "Jewish_ancestry", "Hispanic_or_Latino", "How_often_do_you_eat_POULTRY_BEEF_PORK_SEAFOOD_EGGS",
    "Day_of_stool_collection_diarrhea", "Race", "IBS", "BMI", "Intestinal_disease", "Probiotic",
    "Antihistamines", "Gained_10lbs_in_last_year", "Celiac_disease", "Bristol_stool_chart",
    "How_often_do_you_eat_FRUITS_or_VEGETABLES", "Do_you_drink_caffeinated_beverages",
    "How_often_do_you_eat_GRAINS", "How_often_do_you_eat_YOGURT", "Crohns_disease",
    "Day_of_stool_collection_bloating", "Antibiotics_past_3_months"
]

# 각 컬럼에 대해 unique 값과 개수 출력
for col in target_cols:
    if col in df.columns:
        unique_vals = df[col].dropna().unique()
        print(f"--- {col} (Unique count: {len(unique_vals)}) ---")
        for val in unique_vals:
            print(val)
        print("\n")
    else:
        print(f"[!] Column not found: {col}\n")


--- Sleep_aid (Unique count: 2) ---
Y
N


--- Pain_med (Unique count: 2) ---
N
Y


--- Ulcer_past_3_months (Unique count: 2) ---
N
Y


--- SIBO (Unique count: 2) ---
N
Y


--- Day_of_stool_collection_digestion_issue (Unique count: 2) ---
N
Y


--- Cholesterol_med (Unique count: 2) ---
N
Y


--- Antibiotics_current (Unique count: 2) ---
N
Y


--- Constipation (Unique count: 2) ---
Y
N


--- How_often_do_you_eat_NUTS (Unique count: 4) ---
Few times a month
Less than once a month or never
At least once a day
Few times a week


--- Indigestion_drugs (Unique count: 2) ---
Y
N


--- Asthma_or_COPD_med (Unique count: 2) ---
N
Y


--- Thyroid_med (Unique count: 2) ---
N
Y


--- Diarrhea (Unique count: 2) ---
N
Y


--- GI_cancer_past_3_months (Unique count: 2) ---
N
Y


--- Anti_inflammatory_drugs (Unique count: 2) ---
N
Y


--- Day_of_stool_collection_abdominal_pain (Unique count: 2) ---
N
Y


--- Day_of_stool_collection_excess_gas (Unique count: 2) ---
N
Y


--- Co_Q_10 (Unique count: 2) ---


In [15]:
import pandas as pd

# CSV 불러오기
df = pd.read_csv("all_column_delete7.csv")

# 대상 컬럼 목록
target_cols = [
    "Sleep_aid", "Pain_med", "Ulcer_past_3_months", "SIBO", "Day_of_stool_collection_digestion_issue",
    "Cholesterol_med", "Antibiotics_current", "Constipation", "How_often_do_you_eat_NUTS",
    "Indigestion_drugs", "Asthma_or_COPD_med", "Thyroid_med", "Diarrhea", "GI_cancer_past_3_months",
    "Anti_inflammatory_drugs", "Day_of_stool_collection_abdominal_pain", "Day_of_stool_collection_excess_gas",
    "Co_Q_10", "Do_you_drink_alcohol", "Radiation_Chemo", "Blood_thinners", "Diabetes_med", "Laxatives",
    "Colitis", "Depression_anxiety_mood_med", "Birth_control_or_estrogen", "Blood_pressure_med",
    "Day_of_stool_collection_constipation", "Loss_10lbs_in_last_year", "Do_you_smoke", "IBD",
    "Jewish_ancestry", "Hispanic_or_Latino", "How_often_do_you_eat_POULTRY_BEEF_PORK_SEAFOOD_EGGS",
    "Day_of_stool_collection_diarrhea", "Race", "IBS", "BMI", "Intestinal_disease", "Probiotic",
    "Antihistamines", "Gained_10lbs_in_last_year", "Celiac_disease", "Bristol_stool_chart",
    "How_often_do_you_eat_FRUITS_or_VEGETABLES", "Do_you_drink_caffeinated_beverages",
    "How_often_do_you_eat_GRAINS", "How_often_do_you_eat_YOGURT", "Crohns_disease",
    "Day_of_stool_collection_bloating", "Antibiotics_past_3_months"
]


# 결측치 비율 계산 및 출력
print("--- Missing Value Ratio by Column ---")
for col in target_cols:
    if col in df.columns:
        missing_ratio = df[col].isna().mean()
        print(f"{col}: {missing_ratio:.2%}")
    else:
        print(f"[!] Column not found: {col}")


--- Missing Value Ratio by Column ---
Sleep_aid: 55.85%
Pain_med: 55.53%
Ulcer_past_3_months: 55.15%
SIBO: 56.11%
Day_of_stool_collection_digestion_issue: 59.22%
Cholesterol_med: 55.41%
Antibiotics_current: 55.09%
Constipation: 55.85%
How_often_do_you_eat_NUTS: 54.90%
Indigestion_drugs: 55.73%
Asthma_or_COPD_med: 55.47%
Thyroid_med: 55.34%
Diarrhea: 56.23%
GI_cancer_past_3_months: 56.49%
Anti_inflammatory_drugs: 56.04%
Day_of_stool_collection_abdominal_pain: 57.57%
Day_of_stool_collection_excess_gas: 57.70%
Co_Q_10: 55.28%
Do_you_drink_alcohol: 55.22%
Radiation_Chemo: 55.47%
Blood_thinners: 55.60%
Diabetes_med: 55.60%
Laxatives: 55.41%
Colitis: 55.47%
Depression_anxiety_mood_med: 55.34%
Birth_control_or_estrogen: 56.04%
Blood_pressure_med: 55.34%
Day_of_stool_collection_constipation: 59.35%
Loss_10lbs_in_last_year: 54.96%
Do_you_smoke: 55.28%
IBD: 55.66%
Jewish_ancestry: 56.68%
Hispanic_or_Latino: 55.34%
How_often_do_you_eat_POULTRY_BEEF_PORK_SEAFOOD_EGGS: 54.83%
Day_of_stool_collectio

11.
title(일부 데이터에 시계열 정보), organism (동일한 단일 값) -> 제거

In [1]:
import pandas as pd

# CSV 불러오기
df = pd.read_csv("all_column_delete10.csv")

# 삭제할 칼럼 목록
columns_to_drop = ["title", "organism"]

# 존재하는 컬럼만 삭제
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

# 결과 저장
df.to_csv("all_column_delete11.csv", index=False)


## 그 다음에 이후에 칼럼별 전처리 -> 결측의 채우는 행위, 피쳐엔지니어링 등 


parkinson 칼럼 전처리 1,0 , 나머지 경우